<a href="https://colab.research.google.com/github/vishallmaurya/sentiment_analyzer_backend/blob/main/sentiment_analysis3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 32.2 MB/s eta 0:00:00


In [5]:
import numpy as np
import pandas as pd
from collections import Counter
import pickle
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import SGDClassifier
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import PassiveAggressiveClassifier
import sys
import emoji
from text_preprocessing import cleaning_tweets


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
data = pd.read_csv('/content/sentiment_data_2_label_annotated.csv')
data1 = pd.read_csv('/content/sentiment_data_3csv_annotated.csv')
data2 = pd.read_csv('/content/sentiment_data_label_annotated.csv')
# data3 = pd.read_csv('/content/synthetic_sentiment_data.csv')

In [7]:
data = pd.concat([data, data1, data2])

In [8]:
data.drop(['Unnamed: 0', 'Date', 'class'], inplace=True, axis=1)

In [9]:
cleaned_tweets = []
emoji_list = []

for i in range(data.shape[0]):
  clean_text, extracted_emojis = cleaning_tweets(str(data.iloc[i]["Tweet"]))
  cleaned_tweets.append(clean_text)
  emoji_list.append(extracted_emojis)

data["clean_tweet"] = cleaned_tweets
data["extracted_emojis"] = emoji_list

In [10]:
model = SentenceTransformer("all-MiniLM-L6-v2")

def emoji_to_text(emoji_string):
    return ' '.join(emoji.demojize(e) for e in emoji_string) if emoji_string else ''

data["emoji_text"] = data["extracted_emojis"].apply(emoji_to_text)

text_embeddings = model.encode(data["clean_tweet"].tolist(), convert_to_numpy=True)
data["text_embeddings"] = list(text_embeddings)

embedding_dim = model.get_sentence_embedding_dimension()

emoji_embeddings = [
    model.encode(emoji_text, convert_to_numpy=True) if emoji_text.strip() else np.zeros(embedding_dim)
    for emoji_text in data["emoji_text"]
]

data["emoji_embeddings"] = list(emoji_embeddings)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
from imblearn.over_sampling import SMOTE
smote_balance = SMOTE()

X_text = np.array(data["text_embeddings"].tolist())
X_emoji = np.array(data["emoji_embeddings"].tolist())


X_final = np.hstack((X_text, X_emoji))


y = data["gpt_label"].values

smote = SMOTE()
X_SMOTE, Y_SMOTE = smote.fit_resample(X_final, y)

In [12]:

X_train1, X_test1, y_train1, y_test1 = train_test_split(X_SMOTE, Y_SMOTE, test_size=0.2, random_state=42)


In [13]:
incremental_clf = SGDClassifier(loss="perceptron", warm_start=True, class_weight="balanced")
incremental_clf.fit(X_train1, y_train1)

SGDClassifier(class_weight='balanced', loss='perceptron', warm_start=True)

In [14]:
y_pred1 = incremental_clf.predict(X_test1)
print(f"SGD Accuracy: {accuracy_score(y_test1, y_pred1): .4f}" )

SGD Accuracy:  0.8991


In [ ]:
pickle.dump(incremental_clf, open("model.pkl", "wb"))
pickle.dump(model, open("vectorizer.pkl", "wb"))